# IBIT Backtest: AMMA vs Buy & Hold vs Trend+Mining+OU

This notebook runs three equity curves on IBIT-aligned dates:
- **AMMA** (from `Crypto.Backtest.amma`)
- **Buy & Hold (IBIT)**
- **Trend + Mining + OU** (mining cost sourced from `cleaned_crypto_data.csv`)

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from Crypto.Backtest.amma import amma_from_ibit_csv
from Crypto.Backtest.metrics import build_summary_table
from Crypto.Data.raw_data_loader import load_ibit_with_mining_cost
from Crypto.Models.trend import trend_signal
from Crypto.Models.mining import mining_signal
from Crypto.Models.ou import ou_signal

plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
IBIT_PATH = Path('Crypto/Data/IBIT_Data.csv')
CLEANED_PATH = Path('Crypto/Data/cleaned_crypto_data.csv')

merged = load_ibit_with_mining_cost(
    ibit_path=str(IBIT_PATH),
    cleaned_crypto_path=str(CLEANED_PATH),
    forward_fill_mining_cost=True,
)

merged['ret'] = merged['close'].pct_change().fillna(0.0)
merged.head()

In [ ]:
# 1) AMMA strategy
amma = amma_from_ibit_csv(
    ibit_csv_path=str(IBIT_PATH),
    momentum_weights={5: 0.25, 20: 0.35, 60: 0.40},
    threshold=0.0,
    long_enabled=True,
    short_enabled=False,
)
amma = amma.set_index('date')
amma = amma.loc[merged.index.min():merged.index.max()].copy()
amma.head()

In [ ]:
# 2) Trend + Mining + OU combined strategy
model_df = pd.DataFrame(index=merged.index)
model_df['BTC-USD_close'] = merged['close']
model_df['COST_TO_MINE'] = merged['mining_cost']

trend_pos = trend_signal(
    model_df,
    price_column='BTC-USD_close',
    fast_window=20,
    slow_window=128,
    long_only=True,
)

mining_pos = mining_signal(
    model_df,
    z_window=180,
    entry_z=0.5,
    exit_z=0.0,
    use_log_edge=True,
)

ou_pos = ou_signal(
    model_df['BTC-USD_close'],
    window=90,
    entry_z=1.5,
    exit_z=0.3,
    long_short=False,
    detrend_window=180,
)

# Explicit combine rule: convert each to {0,1}, then majority vote long if >=2
trend_sig = (pd.Series(trend_pos, index=model_df.index) > 0).astype(float)
mining_sig = (pd.Series(mining_pos, index=model_df.index) > 0).astype(float)
ou_sig = (pd.Series(ou_pos, index=model_df.index) > 0).astype(float)

combo_score = trend_sig + mining_sig + ou_sig
combo_signal = (combo_score >= 2).astype(float)
combo_strategy_ret = combo_signal.shift(1).fillna(0.0) * merged['ret']
combo_equity = (1.0 + combo_strategy_ret).cumprod()

In [ ]:
# 3) Buy & Hold and alignment
bh_ret = merged['ret']
bh_equity = (1.0 + bh_ret).cumprod()

equity_df = pd.concat([
    amma['equity'].rename('Amma'),
    bh_equity.rename('Buy & Hold (IBIT)'),
    combo_equity.rename('Trend+Mining+OU'),
], axis=1, join='inner').dropna()

equity_df.tail()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(equity_df.index, equity_df['Amma'], label='Amma', linewidth=2)
plt.plot(equity_df.index, equity_df['Buy & Hold (IBIT)'], label='Buy & Hold (IBIT)', linewidth=2)
plt.plot(equity_df.index, equity_df['Trend+Mining+OU'], label='Trend+Mining+OU', linewidth=2)
plt.title('IBIT Equity Curve Comparison')
plt.xlabel('Date')
plt.ylabel('Equity (Start = 1.0)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
summary = build_summary_table({
    'Amma': {
        'returns': amma.loc[equity_df.index, 'strategy_ret'],
        'position': amma.loc[equity_df.index, 'signal'],
    },
    'Buy & Hold (IBIT)': {
        'returns': bh_ret.loc[equity_df.index],
        'position': pd.Series(1.0, index=equity_df.index),
    },
    'Trend+Mining+OU': {
        'returns': combo_strategy_ret.loc[equity_df.index],
        'position': combo_signal.loc[equity_df.index],
    },
})
summary